In [16]:

import numpy as np 
import math
import time
from math import exp, log, pi, sqrt
from scipy import stats

In [98]:
class OptionsPricing:

    def __init__(self, S, K, T, r, q, sigma):
        self.S = S
        self.K = K
        self.T = T
        self.r = r
        self.q = q 
        self.sigma = sigma 

#================================ CALLS ====================================================================

    def call_price(self):

        d1 = (np.log(S/K) + t * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(t))
        d2 = d1 - sigma * sqrt(t)

        price = S * np.exp(-q * t) * stats.norm.cdf(d1) - K * np.exp(-r * t) * stats.norm.cdf(d2)

        return price

    def call_delta(self):

        d1 = (np.log(S/K) + t * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(t))
        d2 = d1 - sigma * sqrt(t)

        delta = stats.norm.cdf(d1) * exp(-q * T)

        return delta

    def call_gamma(self):

        d1 = (np.log(S/K) + t * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(t))
        d2 = d1 - sigma * sqrt(t)

        gamma = (exp(-1* d1 ** 2) / 2) / sqrt(2 * pi)

        return gamma

    def call_theta(self):

        d1 = (np.log(S/K) + t * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(t))
        d2 = d1 - sigma * sqrt(t)

        theta = (-(S * exp(-1 * (d1 ** 2) / 2) / sqrt(2 * pi) * sigma * exp(-q * T)/ (2 * sqrt(T))) - (r * (K * exp(-r * T)) * stats.norm.cdf(d2)) + (q * S * stats.norm.cdf(d1) * exp(-q * T))) / 365      #Theta defined as the loss of value every calendar day (not trading day)

        return theta

    def call_vega(self):

        d1 = (np.log(S/K) + t * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(t))
        d2 = d1 - sigma * sqrt(t)

        vega = exp(-1 * d1 ** 2 / 2) / sqrt(2 * pi) * exp(-q * T) * S * sqrt(T) / 100

        return vega
    
    def call_rho(self):

        d1 = (np.log(S/K) + t * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(t))
        d2 = d1 - sigma * sqrt(t)

        rho = K * T * exp(-r * T) * stats.norm.cdf(d2) / 100  

        return rho


#================================ PUTS ====================================================================

    def put_price(self):

        d1 = (np.log(S/K) + t * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(t))
        d2 = d1 - sigma * sqrt(t)

        price = K * np.exp(-r * t) * stats.norm.cdf(-d2) - S * np.exp(-q * t) * stats.norm.cdf(-d1)

        return price

    def put_delta(self):

        d1 = (np.log(S/K) + t * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(t))
        d2 = d1 - sigma * sqrt(t)

        delta = (stats.norm.cdf(d1)-1) * exp(-q * T)

        return delta

    def put_gamma(self):

        d1 = (np.log(S/K) + t * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(t))
        d2 = d1 - sigma * sqrt(t)

        gamma = (exp(-1* d1 ** 2) / 2) / sqrt(2 * pi)

        return gamma

    def put_theta(self):

        d1 = (np.log(S/K) + t * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(t))
        d2 = d1 - sigma * sqrt(t)

        theta = (-(S * exp(-1 * d1 ** 2 / 2) / sqrt(2 * pi) * sigma * exp(-q * T) / (2 * sqrt(T))) + (r * K * exp(-r * T) * stats.norm.cdf(-d2)) - (q * S * stats.norm.cdf(-d1) * exp(-q * T))) / 365         # Theta defined as the loss of value every calendar day (365), not every trading day

        return theta

    def put_vega(self):

        d1 = (np.log(S/K) + t * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(t))
        d2 = d1 - sigma * sqrt(t)

        vega = exp(-1 * d1 ** 2 / 2) / sqrt(2 * pi) * exp(-q * T) * S * sqrt(T) / 100

        return vega

    def put_rho(self):

        d1 = (np.log(S/K) + t * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(t))
        d2 = d1 - sigma * sqrt(t)

        rho = -K * T * exp(-r * T) * stats.norm.cdf(-d2) / 100

        return rho


In [99]:
S = 65.24
K = 65
r = 0.01
q = 0.00
T = 15/365
sigma = 0.3955

In [100]:
model = OptionsPricing(S, K, T, r, q, sigma)

callprice = model.call_price()
calldelta = model.call_delta()
callgamma = model.call_gamma()
calltheta = model.call_theta()
callvega = model.call_vega()
callrho = model.call_rho()

putprice = model.put_price()
putdelta = model.put_delta()
putgamma = model.call_gamma()
puttheta = model.put_theta()
putvega = model.put_vega()
putrho = model.put_rho()


In [102]:
putrho

-0.01450707906058311